In [1]:
import pymongo
import shapely
import folium
import copy
import random

In [2]:
import pandas as pd
import pymongo
import geopandas as gpd

myclient = pymongo.MongoClient("mongodb://192.168.120.127:27017")
mydb = myclient["tmap"]
mylink = mydb["link1"]
data = pd.DataFrame(list(mydb.link1.find()))
data.head()

,_added_direction,_id,_nodes,_nodes1,access,bridge,geometry_geojson,highway,id1,id2,...,length,maxspeed,name,oneway,osmid,ref,tunnel,u,v,width
0,False,5c2f6ecbf0c307b60d9c0c9f,"[322043913, 322044026]","[322043913, 322044026]",NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.05...",unclassified,40DgV7c7kWj5nsex9JKo,MOBVA47tVlowW8,...,58.294,NaN,NaN,False,29285059,NaN,NaN,322043913,322044026,NaN
1,False,5c2f6ecbf0c307b60d9c0ca0,"[322043913, 5346586516, 5180292950, 5180292951...","[322043913, 322043886]",NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.05...",tertiary,vP8yxWlTZekMYRsBrAOVb,2WyL3dvHErLYl2fpZ3VEnfvOeKZviwJB4gdCAWEZxM,...,376.965,NaN,杨梅山路,False,553954463,NaN,NaN,322043913,322043886,NaN
2,True,5c2f6ecbf0c307b60d9c0ca1,"[322043913, 322043923, 3601760669, 5346586515,...","[322043913, 3601760665]",NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.05...",tertiary,e8y4nevS0LlVbKsAZldwq,vxBqZ5bcZekvKncPQ0MDBTKxdAJ1cqKO6b2ik4Vo1qH2Bz...,...,633.995,NaN,杨梅山路,False,553954463,NaN,NaN,322043913,3601760665,NaN
3,True,5c2f6ecbf0c307b60d9c0ca2,"[5137214804, 5214510869]","[5137214804, 5214510869]",NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.13...",unclassified,VK3kwRbcLoPx25HxDZoeo,RlRp6O7FEWz8q8,...,11.532,NaN,风景街,False,528685411,NaN,NaN,5137214804,5214510869,NaN
4,False,5c2f6ecbf0c307b60d9c0ca3,"[5137214804, 5217977962, 5217977961]","[5137214804, 5217977961]",NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.13...",secondary,gWZYrlKInk09yACO2zkXq,7MyOgelH0MbRkeCg0vwxD,...,92.753,NaN,小河路,True,421596132,NaN,NaN,5137214804,5217977961,NaN


In [27]:
data1 = pd.DataFrame(list(mylink.find()))
data1 = data1.sort_values(by=['length'], ascending=False)
data1.head()

,_added_direction,_id,_nodes,_nodes1,access,bridge,geometry_geojson,highway,id1,id2,...,length,maxspeed,name,oneway,osmid,ref,tunnel,u,v,width
13550,False,5c2e611a0ed16e3189ddc7d6,"[5160236076, 5160236116, 5160236115, 516023611...","[5160236076, 5158768814]",NaN,NaN,"{'coordinates': [[120.3539126, 30.2136184], [1...",primary,lWqevL5hW1ekKRU2rBAA0,wdJB97wI0keOvWTJ3EK5Kc1O4RDzTroVkADIKd0MQqtzX8...,...,12338.754,NaN,机场城市大道,True,531431468,NaN,NaN,5160236076,5158768814,NaN
12667,False,5c2e611a0ed16e3189ddc463,"[4820100209, 5158768808, 5158768809, 515876881...","[4820100209, 5160236078]",NaN,NaN,"{'coordinates': [[120.2279977, 30.2198489], [1...",primary,E9EQoM2tr0A5YzHBzkLR4,jKX5dp9h9oE11ytY5drrWfo0y448UL24OOLiqPxQQwI73g...,...,12221.662,NaN,机场城市大道,True,531431467,NaN,NaN,4820100209,5160236078,NaN
12322,False,5c2e611a0ed16e3189ddc30a,"[3035358452, 3035357588, 5596048378, 289068070...","[3035358452, 2890680730]",NaN,NaN,"{'coordinates': [[120.3344389, 30.2006057], [1...",primary,7nQpJOoCjlWo9RUon4xD6,QVJq6QKInqyeDXiz6QE4VCjE6J0eFxn57DBco75xwKh34A...,...,9990.653,NaN,建设四路,True,505097816,NaN,NaN,3035358452,2890680730,NaN
2301,False,5c2e611a0ed16e3189dd9be5,"[4087229825, 5634394752, 1668711812, 289068074...","[4087229825, 1728952765]",NaN,NaN,"{'coordinates': [[120.2304275, 30.2144327], [1...",primary,VdLq3Zf7w3EqkTZMzeE1,13dk25xUEKB2Z8Ul4A5oJUzA4r9rhJ9RJkOtZ3JZ6LTjvb...,...,7967.499,NaN,建设四路,True,113467440,NaN,NaN,4087229825,1728952765,NaN
20652,False,5c2e611a0ed16e3189dde394,"[5598294227, 5956192106, 5541665318, 554166531...","[5598294227, 3373270288]",NaN,yes,"{'coordinates': [[120.2454129, 30.4243771], [1...",trunk,k9ez8vyH7Ol49jHOK1jEy,rBVwljRiwBBRE7fDyQ8lzU5lgnwMfkwRpD0S16OkwPfMEY...,...,6147.281,NaN,秋石高架,True,658403039,NaN,NaN,5598294227,3373270288,NaN


In [6]:
data.columns.values.tolist()

['_added_direction',
 '_id',
 '_nodes',
 '_nodes1',
 'access',
 'bridge',
 'geometry_geojson',
 'highway',
 'id1',
 'id2',
 'junction',
 'key',
 'lanes',
 'length',
 'maxspeed',
 'name',
 'oneway',
 'osmid',
 'ref',
 'tunnel',
 'u',
 'v',
 'width']

In [18]:
import geojson
# data['geometry_geojson'] = data['geometry'].apply(geojson.loads)
data['geometry'] = data['geometry_geojson'].apply(shapely.geometry.asShape)
data = data.drop(columns=['_id'])

In [19]:
gdf1 = gpd.GeoDataFrame(data)
gdf1.head()

,_added_direction,_nodes,_nodes1,access,bridge,geometry_geojson,highway,id1,id2,junction,...,maxspeed,name,oneway,osmid,ref,tunnel,u,v,width,geometry
0,False,"[322043913, 322044026]","[322043913, 322044026]",NaN,NaN,"{'coordinates': [[120.0586011, 30.2389999], [1...",unclassified,40DgV7c7kWj5nsex9JKo,MOBVA47tVlowW8,NaN,...,NaN,NaN,False,29285059,NaN,NaN,322043913,322044026,NaN,"LINESTRING (120.0586011 30.2389999, 120.059172..."
1,False,"[322043913, 5346586516, 5180292950, 5180292951...","[322043913, 322043886]",NaN,NaN,"{'coordinates': [[120.0586011, 30.2389999], [1...",tertiary,vP8yxWlTZekMYRsBrAOVb,2WyL3dvHErLYl2fpZ3VEnfvOeKZviwJB4gdCAWEZxM,NaN,...,NaN,杨梅山路,False,553954463,NaN,NaN,322043913,322043886,NaN,"LINESTRING (120.0586011 30.2389999, 120.058620..."
2,True,"[322043913, 322043923, 3601760669, 5346586515,...","[322043913, 3601760665]",NaN,NaN,"{'coordinates': [[120.0586011, 30.2389999], [1...",tertiary,e8y4nevS0LlVbKsAZldwq,vxBqZ5bcZekvKncPQ0MDBTKxdAJ1cqKO6b2ik4Vo1qH2Bz...,NaN,...,NaN,杨梅山路,False,553954463,NaN,NaN,322043913,3601760665,NaN,"LINESTRING (120.0586011 30.2389999, 120.058507..."
3,True,"[5137214804, 5214510869]","[5137214804, 5214510869]",NaN,NaN,"{'coordinates': [[120.1321243, 30.3236806], [1...",unclassified,VK3kwRbcLoPx25HxDZoeo,RlRp6O7FEWz8q8,NaN,...,NaN,风景街,False,528685411,NaN,NaN,5137214804,5214510869,NaN,"LINESTRING (120.1321243 30.3236806, 120.132004..."
4,False,"[5137214804, 5217977962, 5217977961]","[5137214804, 5217977961]",NaN,NaN,"{'coordinates': [[120.1321243, 30.3236806], [1...",secondary,gWZYrlKInk09yACO2zkXq,7MyOgelH0MbRkeCg0vwxD,NaN,...,NaN,小河路,True,421596132,NaN,NaN,5137214804,5217977961,NaN,"LINESTRING (120.1321243 30.3236806, 120.132104..."


In [20]:
tips = folium.features.GeoJsonTooltip(fields=['name'])

In [21]:
highlight_function = lambda x:{'color': "#000000"}

In [22]:
color_func = lambda x:{'color': "#%06x" % random.randint(0, 0xFFFFFF),'opacity':0.5}

In [25]:
gdf2 = gdf1.iloc[:100]

In [31]:
m = folium.Map([30.186518,120.176028], zoom_start=13)

folium.GeoJson(gdf2.__geo_interface__,style_function=color_func,tooltip=tips,highlight_function=highlight_function).add_to(m)

m

In [40]:
data1['geometry'] = data['geometry_geojson'].apply(shapely.geometry.asShape)
# data1 = data1.drop(columns=['_id'])

gdf3 = gpd.GeoDataFrame(data1)

tips = folium.features.GeoJsonTooltip(fields=['name'])
highlight_function = lambda x:{'color': "#000000"}
color_func = lambda x:{'color': "#%06x" % random.randint(0, 0xFFFFFF),'opacity':0.5}

gdf4 = gdf3.head(100)

m2 = folium.Map([30.186588,120.176088], zoom_start=13)

folium.GeoJson(gdf4.__geo_interface__,style_function=color_func,tooltip=tips,highlight_function=highlight_function).add_to(m)

m2

In [37]:
gdf4

,_added_direction,_nodes,_nodes1,access,bridge,geometry_geojson,highway,id1,id2,junction,...,maxspeed,name,oneway,osmid,ref,tunnel,u,v,width,geometry
13550,False,"[5160236076, 5160236116, 5160236115, 516023611...","[5160236076, 5158768814]",NaN,NaN,"{'coordinates': [[120.3539126, 30.2136184], [1...",primary,lWqevL5hW1ekKRU2rBAA0,wdJB97wI0keOvWTJ3EK5Kc1O4RDzTroVkADIKd0MQqtzX8...,NaN,...,NaN,机场城市大道,True,531431468,NaN,NaN,5160236076,5158768814,NaN,"LINESTRING (120.3539126 30.2136184, 120.353324..."
12667,False,"[4820100209, 5158768808, 5158768809, 515876881...","[4820100209, 5160236078]",NaN,NaN,"{'coordinates': [[120.2279977, 30.2198489], [1...",primary,E9EQoM2tr0A5YzHBzkLR4,jKX5dp9h9oE11ytY5drrWfo0y448UL24OOLiqPxQQwI73g...,NaN,...,NaN,机场城市大道,True,531431467,NaN,NaN,4820100209,5160236078,NaN,"LINESTRING (120.2279977 30.2198489, 120.228171..."
12322,False,"[3035358452, 3035357588, 5596048378, 289068070...","[3035358452, 2890680730]",NaN,NaN,"{'coordinates': [[120.3344389, 30.2006057], [1...",primary,7nQpJOoCjlWo9RUon4xD6,QVJq6QKInqyeDXiz6QE4VCjE6J0eFxn57DBco75xwKh34A...,NaN,...,NaN,建设四路,True,505097816,NaN,NaN,3035358452,2890680730,NaN,"LINESTRING (120.3344389 30.2006057, 120.334301..."
2301,False,"[4087229825, 5634394752, 1668711812, 289068074...","[4087229825, 1728952765]",NaN,NaN,"{'coordinates': [[120.2304275, 30.2144327], [1...",primary,VdLq3Zf7w3EqkTZMzeE1,13dk25xUEKB2Z8Ul4A5oJUzA4r9rhJ9RJkOtZ3JZ6LTjvb...,NaN,...,NaN,建设四路,True,113467440,NaN,NaN,4087229825,1728952765,NaN,"LINESTRING (120.2304275 30.2144327, 120.230672..."
20652,False,"[5598294227, 5956192106, 5541665318, 554166531...","[5598294227, 3373270288]",NaN,yes,"{'coordinates': [[120.2454129, 30.4243771], [1...",trunk,k9ez8vyH7Ol49jHOK1jEy,rBVwljRiwBBRE7fDyQ8lzU5lgnwMfkwRpD0S16OkwPfMEY...,NaN,...,NaN,秋石高架,True,658403039,NaN,NaN,5598294227,3373270288,NaN,"LINESTRING (120.2454129 30.4243771, 120.236819..."
272,False,"[3373270289, 5157012938, 5157012939, 515701294...","[3373270289, 5598294232]",NaN,yes,"{'coordinates': [[120.1947738, 30.3946484], [1...",trunk,Pn35LJgU9MxPgRcERLKX4,zPLbDkOTyEdPRZI702M34fvwDB6es1DbgzYUMzzvdPFqLL...,NaN,...,NaN,秋石高架,True,531234440,NaN,NaN,3373270289,5598294232,NaN,"LINESTRING (120.1947738 30.3946484, 120.195389..."
23022,False,"[1313052137, 271899651, 271899728, 271899531, ...","[1313052137, 3752945758]",NaN,NaN,"{'coordinates': [[120.0464028, 30.2345167], [1...",motorway,JQv00euz3RpxZT5PglQ7,1WVED59fAMYXAYhXxjozpC71AjKPSqkDAVVid64zqWuEov...,NaN,...,NaN,绕城高速 Ràochéng Gāosù,True,116484186,G25 G2501,NaN,1313052137,3752945758,NaN,"LINESTRING (120.0464028 30.2345167, 120.055701..."
20406,False,"[3752945848, 269972864, 269972870, 269972872, ...","[3752945848, 1313051726]",NaN,NaN,"{'coordinates': [[120.0524295, 30.195263], [12...",motorway,zX3qLDqIZe4nOrH1XrPYM,nbvdK2YCkDk2yOF2M2dyBCpbpzQ8C6O65KqSkokjDyiQRQ...,NaN,...,NaN,绕城高速,True,347589076,G25 G2501,NaN,3752945848,1313051726,NaN,"LINESTRING (120.0524295 30.195263, 120.0524781..."
595,False,"[1899023880, 319288804, 319288780, 3521062365,...","[1899023880, 3521062357]",NaN,NaN,"{'coordinates': [[120.3912397, 30.3841478], [1...",motorway,M2W30QltrpMZY3HEdgZ6E,EdXp73qFzBYnxkh8QR3ZRULxqVy9fxAnORXfzLdYBqcXMkJ0X,NaN,...,NaN,沪昆高速,True,345412046,G60,NaN,1899023880,3521062357,NaN,"LINESTRING (120.3912397 30.3841478, 120.380925..."
19816,False,"[1979801234, 31976787, 31976769, 31976762, 319...","[1979801234, 31976594]",NaN,yes,"{'coordinates': [[120.1164438, 30.3687163], [1...",trunk,PywJ0kVcOkL9DDI9PJ46,3nDkewwtQrex3idZVX9cp1qyMCwzjnOs0bRX2UWXboOiXy...,NaN,...,NaN,上塘高架,True,187247184,NaN,NaN,1979801234,31976594,NaN,"LINESTRING (120.1164438 30.3687163, 120.116597..."
